# QuantLib

In [ ]:
import QuantLib as ql
import datetime as dt

## A simple calendar sample

In [ ]:
nyse = ql.UnitedStates(ql.UnitedStates.NYSE)
fra = ql.Germany(ql.Germany.FrankfurtStockExchange)

In [ ]:
first_date = ql.Date(1,1,2016)
second_date = ql.Date(1,6,2016)

In [ ]:
nyse.businessDaysBetween(first_date, second_date)

In [ ]:
fra.businessDaysBetween(first_date, second_date)

## Option Pricing

In [ ]:
today = dt.datetime(2016, 5, 30)
settlement = dt.datetime(2016, 6, 1)
expiry = dt.datetime(2016, 9, 16)
underlying = 100.35
riskfree = 0.05
vol = 0.20
dividend_yield = 0.0
option_type = 'call'
strike = 100

### Transform Dates

In [ ]:
today = ql.Date(today.day, today.month, today.year)
settlement = ql.Date(settlement.day, settlement.month, settlement.year)
expiry = ql.Date(expiry.day, expiry.month, expiry.year)
ql.Settings.instance().evaluationDate = today

### Market Data

In [ ]:
underlying = ql.SimpleQuote(underlying)
r = ql.SimpleQuote(riskfree)
q = ql.SimpleQuote(dividend_yield)
vol = ql.SimpleQuote(vol)
dcc = ql.Actual365Fixed()

risk_free_curve = ql.FlatForward(settlement, ql.QuoteHandle(r), dcc)
div_yield_curve = ql.FlatForward(settlement, ql.QuoteHandle(q), dcc)
vol_curve = ql.BlackConstantVol(today, ql.UnitedStates(ql.UnitedStates.NYSE), ql.QuoteHandle(vol), dcc)

### Instantiating the option

In [ ]:
exercise = ql.AmericanExercise(settlement, expiry)
if option_type == 'call':
    payoff = ql.PlainVanillaPayoff(type=ql.Option.Call, strike=strike)
elif option_type == 'put':
    payoff = ql.PlainVanillaPayoff(type=ql.Option.Put, strike=strike)

option = ql.VanillaOption(payoff, exercise)

### Price process

In [ ]:
process = ql.BlackScholesMertonProcess(s0=ql.QuoteHandle(underlying),
                                       dividendTS=ql.YieldTermStructureHandle(div_yield_curve),
                                       riskFreeTS=ql.YieldTermStructureHandle(risk_free_curve),
                                       volTS=ql.BlackVolTermStructureHandle(vol_curve)
                                       )

### Calculation Methods
* **Analytical**

In [ ]:
option.setPricingEngine(ql.BaroneAdesiWhaleyEngine(process))
option.NPV()

* **Binomial (Jarrow-Rudd)**

In [ ]:
option.setPricingEngine(ql.BinomialVanillaEngine(process, type='jr', steps=801))
option.NPV()

 **... and many more:**

* Black-Scholes (for european options only)
* Barone-Adesi/Whaley (american-only)
* Bjerksund/Stensland (american)
* Integral (european)
* Finite differences
* Binomial Jarrow-Rudd
* Binomial Cox-Ross-Rubinstein
* Additive equiprobabilities
* Binomial Trigeorgis
* Binomial Tian
* Binomial Leisen-Reimer
* crude Monte Carlo (european-only)
* Sobol-sequence Monte Carlo (european-only)